### Initialization

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

fpath1 = '/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ'
fpath2 ='/home/8ididata/2022-1/babnigg202203/cluster_results_QZ'



### Averaging of SA-XPCS results, temperature ramp up

In [2]:

avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='H041',
                                              num_sections=20,
                                              prefix=fpath1, 
                                              zone_idx='auto',
                                              num_cores=24)


2023-12-20 16:40:28,435 utils                   : total number of files in H041  is 1450
2023-12-20 16:40:29,281 utils                   : Failed to get temperature for file /home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ/H041_VPAVG_H05_2_500C10p_att00_Rq0_01270_0001-100000.hdf, return nan
2023-12-20 16:40:30,237 xpcs_viewer.fileIO.hdf_reader: key not found: /exchange/partition-mean-total
2023-12-20 16:40:30,246 utils                   : failed to read file /home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ/H041_VPAVG_H05_2_500C10p_att00_Rq0_01270_0001-100000.hdf, skip this file
2023-12-20 16:40:31,491 utils                   : label='H041_section_01': remove 8 datasets out of 73
2023-12-20 16:40:31,511 utils                   : label='H041_section_00': remove 10 datasets out of 73
2023-12-20 16:40:31,552 utils                   : label='H041_section_02': remove 9 datasets out of 73
2023-12-20 16:40:31,583 utils                   : label='H041_section_03': remove 8 datas

### Averaging of SA-XPCS results, temperature ramp down

In [3]:

avg_ramp_down, t_el, ql_dyn, ql_sta = process_group(group='H042',
                                              num_sections=20,
                                              prefix=fpath1, 
                                              zone_idx='auto',
                                              num_cores=24)

2023-12-20 16:40:33,452 utils                   : total number of files in H042  is 1450
2023-12-20 16:40:36,425 utils                   : label='H042_section_00': remove 10 datasets out of 73
2023-12-20 16:40:36,426 utils                   : label='H042_section_01': remove 11 datasets out of 73
2023-12-20 16:40:36,443 utils                   : label='H042_section_02': remove 10 datasets out of 73
2023-12-20 16:40:36,448 utils                   : label='H042_section_03': remove 9 datasets out of 73
2023-12-20 16:40:36,530 utils                   : label='H042_section_04': remove 11 datasets out of 73
2023-12-20 16:40:36,547 utils                   : label='H042_section_06': remove 10 datasets out of 73
2023-12-20 16:40:36,547 utils                   : label='H042_section_05': remove 9 datasets out of 73
2023-12-20 16:40:36,563 utils                   : label='H042_section_10': remove 11 datasets out of 72
2023-12-20 16:40:36,562 utils                   : label='H042_section_11': remove

### Averaging of background 

In [4]:

avg_bg, _, _, _ = process_group(group='D029',
                                 num_sections=1, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-20 16:40:38,191 utils                   : total number of files in D029  is 200
2023-12-20 16:40:42,629 utils                   : label='D029_section_00': remove 26 datasets out of 200


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [5]:

fname = 'E0110_S3_4_Ramp_340C10p_att00_Rq0_00001_0001-100000.hdf'

with h5py.File(fpath2+'/'+fname, 'r') as f:
    _ = f['measurement']['sample']['absolute_cross_section_scale'][()]
    abs_coeff = _[0][0]

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = abs_coeff*(avg_ramp_up[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = abs_coeff*(avg_ramp_down[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])


with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
